# Upload and download YT tables

This notebook contains examples of how to download and upload [tables](https://ytsaurus.tech/docs/en/user-guide/storage/files) on YTsaurus.

1. How to work with tables without schema.
2. How to create a table with schema and upload data.
3. How to work with structured tables and yt_dataclass objects.
4. How to upload datetime objects to Datetime columns.

In [2]:
from yt import wrapper as yt
from yt import type_info

from datetime import datetime, timedelta
from dataclasses import asdict
import uuid
import time

## Create a base directory for examples

In [4]:
# configure environment to run this notebooks
import uuid
import yt.wrapper as yt

username = yt.get_user_name()
if yt.exists(f"//sys/users/{username}/@user_info/home_path"):
    # prepare working directory on distributed file system
    user_info = yt.get(f"//sys/users/{yt.get_user_name()}/@user_info")
    homedir = user_info["home_path"]
    # find avaliable vm presets
    cpu_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("cpu")] or ["default"]
    h100_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100")]
    h100_8_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100-8")]
    workdir = f"{homedir}/tmp/demo_workdir/{uuid.uuid4().hex}"
else:
    cpu_pool_trees = ["default"]
    h100_pool_trees = ["gpu_h100"]
    h100_8_pool_trees = ["gpu_h100"]
    workdir = f"//tmp/examples/{uuid.uuid4().hex}"

yt.create("map_node", workdir, recursive=True, ignore_existing=True)
print("Current working directory:", workdir)

Current working directory: //home/equal_amethyst_vulture/tmp/demo_workdir/9477b3083fb2412483500355be28d6d8


## Upload and download unstructured data

The easest way to upload data is using `write_table` method of ytClient. `write_table` creates a table without the [schema](https://ytsaurus.tech/docs/en/user-guide/storage/static-schema). `read_table` automatically converts data from a table to primitive python types.

This method is easy to implement, but does not protect against some typical problems:
1. Typos in column names in different records.
2. Unexpected type conversions.
3. Large tables without schemas use resources inefficiently.

In [6]:
records_unstractued = [
    { 
        "field_string": f"string_{i}",
        "field_float": i / 10,
        "field_date": (datetime.now() - timedelta(days=i)).isoformat(),
        "field_uint32": i,
    }
    for i in range(10)
]
print(records_unstractued)

[{'field_string': 'string_0', 'field_float': 0.0, 'field_date': '2025-06-19T19:27:28.904642', 'field_uint32': 0}, {'field_string': 'string_1', 'field_float': 0.1, 'field_date': '2025-06-18T19:27:28.904657', 'field_uint32': 1}, {'field_string': 'string_2', 'field_float': 0.2, 'field_date': '2025-06-17T19:27:28.904665', 'field_uint32': 2}, {'field_string': 'string_3', 'field_float': 0.3, 'field_date': '2025-06-16T19:27:28.904671', 'field_uint32': 3}, {'field_string': 'string_4', 'field_float': 0.4, 'field_date': '2025-06-15T19:27:28.904676', 'field_uint32': 4}, {'field_string': 'string_5', 'field_float': 0.5, 'field_date': '2025-06-14T19:27:28.904682', 'field_uint32': 5}, {'field_string': 'string_6', 'field_float': 0.6, 'field_date': '2025-06-13T19:27:28.904688', 'field_uint32': 6}, {'field_string': 'string_7', 'field_float': 0.7, 'field_date': '2025-06-12T19:27:28.904694', 'field_uint32': 7}, {'field_string': 'string_8', 'field_float': 0.8, 'field_date': '2025-06-11T19:27:28.904699', 'f

In [7]:
unstructured_table_path = f"{workdir}/unstructured_table"
yt.write_table(unstructured_table_path, records_unstractued)

In [8]:
for record in yt.read_table(unstructured_table_path):
    print(record)

{'field_string': 'string_0', 'field_float': 0.0, 'field_date': '2025-06-19T19:27:28.904642', 'field_uint32': 0}
{'field_string': 'string_1', 'field_float': 0.1, 'field_date': '2025-06-18T19:27:28.904657', 'field_uint32': 1}
{'field_string': 'string_2', 'field_float': 0.2, 'field_date': '2025-06-17T19:27:28.904665', 'field_uint32': 2}
{'field_string': 'string_3', 'field_float': 0.3, 'field_date': '2025-06-16T19:27:28.904671', 'field_uint32': 3}
{'field_string': 'string_4', 'field_float': 0.4, 'field_date': '2025-06-15T19:27:28.904676', 'field_uint32': 4}
{'field_string': 'string_5', 'field_float': 0.5, 'field_date': '2025-06-14T19:27:28.904682', 'field_uint32': 5}
{'field_string': 'string_6', 'field_float': 0.6, 'field_date': '2025-06-13T19:27:28.904688', 'field_uint32': 6}
{'field_string': 'string_7', 'field_float': 0.7, 'field_date': '2025-06-12T19:27:28.904694', 'field_uint32': 7}
{'field_string': 'string_8', 'field_float': 0.8, 'field_date': '2025-06-11T19:27:28.904699', 'field_uint

## Create schema and upload unstructured data

In order to use schematized tables we can create a table with the strong [schema](https://ytsaurus.tech/docs/en/user-guide/storage/static-schema) and upload unstructured data.
Schematization allows efficient use of resources and defines the table structure and column types.

In [10]:
schema = yt.schema.TableSchema()
schema.add_column("field_string", type_info.String)
schema.add_column("field_float", type_info.Float)
schema.add_column("field_date", type_info.Datetime)
schema.add_column("field_uint32", type_info.Uint32)

TableSchema({'value': [{'name': 'field_string', 'type_v3': 'string'}, {'name': 'field_float', 'type_v3': 'float'}, {'name': 'field_date', 'type_v3': 'datetime'}, {'name': 'field_uint32', 'type_v3': 'uint32'}], 'attributes': {'strict': True, 'unique_keys': False}})

YTsaurus client doesn't have standard convertion mechanisms for datetime.datetime objects. Datetime columns expect uint32 values, so we have to convert datetime.datetime objects to python's int.

In [12]:
def datetime_to_unixtime(datetime_obj: datetime) -> int:
    # workaround for https://github.com/ytsaurus/ytsaurus/issues/309
    return int(time.mktime(datetime_obj.timetuple()))

In [13]:
records_handmade_schema = [
    { 
        "field_string": f"string_{i}",
        "field_float": i / 10,
        "field_date": datetime_to_unixtime(datetime.now() - timedelta(days=i)),
        "field_uint32": i,
    }
    for i in range(10)
]
print(records_unstractued)

[{'field_string': 'string_0', 'field_float': 0.0, 'field_date': '2025-06-19T19:27:28.904642', 'field_uint32': 0}, {'field_string': 'string_1', 'field_float': 0.1, 'field_date': '2025-06-18T19:27:28.904657', 'field_uint32': 1}, {'field_string': 'string_2', 'field_float': 0.2, 'field_date': '2025-06-17T19:27:28.904665', 'field_uint32': 2}, {'field_string': 'string_3', 'field_float': 0.3, 'field_date': '2025-06-16T19:27:28.904671', 'field_uint32': 3}, {'field_string': 'string_4', 'field_float': 0.4, 'field_date': '2025-06-15T19:27:28.904676', 'field_uint32': 4}, {'field_string': 'string_5', 'field_float': 0.5, 'field_date': '2025-06-14T19:27:28.904682', 'field_uint32': 5}, {'field_string': 'string_6', 'field_float': 0.6, 'field_date': '2025-06-13T19:27:28.904688', 'field_uint32': 6}, {'field_string': 'string_7', 'field_float': 0.7, 'field_date': '2025-06-12T19:27:28.904694', 'field_uint32': 7}, {'field_string': 'string_8', 'field_float': 0.8, 'field_date': '2025-06-11T19:27:28.904699', 'f

In [14]:
handmade_schema_table_path = f"{workdir}/handmade_schema_table"
yt.create_table(handmade_schema_table_path, attributes={"schema": schema.to_yson_type()})

/slot/sandbox/jlab/lib/python42/site-packages/yt/common.py:79: YtDeprecationWarning: create_table is deprecated and will be removed in the next major release, use "create" with "table" type instead
  warnings.warn(message, YtDeprecationWarning)


'46d-1e2c1-24dd0191-ab324e78'

In [15]:
yt.write_table(handmade_schema_table_path, records_handmade_schema)

## `yt.yt_dataclass` obects for schematized data

YTsaurus sdk provides python-native tables representation. We can use `yt_dataclass` to represent the table's schema and to check the correspondence of python types to the table's schema. [Documentation](https://ytsaurus.tech/docs/en/api/python/dataclass)

In [17]:
@yt.yt_dataclass
class Nested:
    a: str
    b: int

@yt.yt_dataclass
class TableRow:
    field_string: str
    field_float: float
    field_datetime: yt.schema.Datetime
    field_uint32: yt.schema.Uint32
    field_nested: Nested

In [18]:
records_dataclasses = [
    TableRow(
        field_string=f"string_{i}", 
        field_float=i / 10,
        field_datetime=datetime_to_unixtime(datetime.now() - timedelta(days=i)),
        field_uint32=i,
        field_nested=Nested(
            a=f"string_{i}",
            b=i,
        )
    )
    for i in range(10)
]
for record in records_dataclasses:
    print(records_dataclasses)

[TableRow(field_string='string_0', field_float=0.0, field_datetime=1750361250, field_uint32=0, field_nested=Nested(a='string_0', b=0)), TableRow(field_string='string_1', field_float=0.1, field_datetime=1750274850, field_uint32=1, field_nested=Nested(a='string_1', b=1)), TableRow(field_string='string_2', field_float=0.2, field_datetime=1750188450, field_uint32=2, field_nested=Nested(a='string_2', b=2)), TableRow(field_string='string_3', field_float=0.3, field_datetime=1750102050, field_uint32=3, field_nested=Nested(a='string_3', b=3)), TableRow(field_string='string_4', field_float=0.4, field_datetime=1750015650, field_uint32=4, field_nested=Nested(a='string_4', b=4)), TableRow(field_string='string_5', field_float=0.5, field_datetime=1749929250, field_uint32=5, field_nested=Nested(a='string_5', b=5)), TableRow(field_string='string_6', field_float=0.6, field_datetime=1749842850, field_uint32=6, field_nested=Nested(a='string_6', b=6)), TableRow(field_string='string_7', field_float=0.7, fie

In [19]:
dataclass_table_path = f"{workdir}/dataclass_based_table"
yt.write_table_structured(dataclass_table_path, TableRow, records_dataclasses)

In [20]:
for record in yt.read_table_structured(dataclass_table_path, TableRow):
    print(record)

TableRow(field_string='string_0', field_float=0.0, field_datetime=1750361250, field_uint32=0, field_nested=Nested(a='string_0', b=0))
TableRow(field_string='string_1', field_float=0.1, field_datetime=1750274850, field_uint32=1, field_nested=Nested(a='string_1', b=1))
TableRow(field_string='string_2', field_float=0.2, field_datetime=1750188450, field_uint32=2, field_nested=Nested(a='string_2', b=2))
TableRow(field_string='string_3', field_float=0.3, field_datetime=1750102050, field_uint32=3, field_nested=Nested(a='string_3', b=3))
TableRow(field_string='string_4', field_float=0.4, field_datetime=1750015650, field_uint32=4, field_nested=Nested(a='string_4', b=4))
TableRow(field_string='string_5', field_float=0.5, field_datetime=1749929250, field_uint32=5, field_nested=Nested(a='string_5', b=5))
TableRow(field_string='string_6', field_float=0.6, field_datetime=1749842850, field_uint32=6, field_nested=Nested(a='string_6', b=6))
TableRow(field_string='string_7', field_float=0.7, field_datet